In [ ]:
import numpy as np 
import pandas as pd 


In [ ]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
print('Done!')

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.shape, news_train_df.shape

In [ ]:
market_train_df.head()

In [ ]:
market_train_df = market_train_df
news_train_df = news_train_df

In [ ]:
# import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
from sklearn.linear_model import SGDRegressor
#from fancyimpute import simple_fill, IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.cluster.bicluster import SpectralCoclustering

from xgboost.sklearn import XGBRegressor, XGBClassifier
from sklearn.linear_model import LogisticRegression

# from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import VotingClassifier

%matplotlib inline

In [ ]:
from itertools import chain

def fast_flatten(input_list):
    return list(chain.from_iterable(input_list))

def fast_concat(frames):
    COLUMN_NAMES = frames[0].columns
    df_dict = dict.fromkeys(COLUMN_NAMES, [])
    for col in COLUMN_NAMES:
        # Use a generator to save memory
        extracted = (frame[col] for frame in frames)

        # Flatten and save to df_dict
        df_dict[col] = fast_flatten(extracted)
    return(pd.DataFrame.from_dict(df_dict)[COLUMN_NAMES])

In [ ]:
news_cols_agg = {
#     'urgency': ['min', 'count'],
    #'takeSequence': ['max'],
    #'bodySize': ['min', 'max', 'mean', 'std'],
    #'wordCount': ['min', 'max', 'mean', 'std'],
    #'sentenceCount': ['min', 'max', 'mean', 'std'],
    #'companyCount': ['min', 'max', 'mean', 'std'],
#     'marketCommentary': ['min', 'max', 'mean', 'std'],
#     'relevance': ['min', 'max', 'mean', 'std'],
    'sentimentNegative': ['min', 'max', 'mean', 'std'],
#     'sentimentNegative': ['median'],
    #'sentimentNeutral': ['min', 'max', 'mean', 'std'],
    'sentimentPositive': ['min', 'max', 'mean', 'std'],
#     'sentimentPositive': ['median'],
    #'sentimentWordCount': ['min', 'max', 'mean', 'std'],
    'noveltyCount12H': ['min', 'max', 'mean', 'std'],
#     'noveltyCount12H': ['median']
    #'noveltyCount24H': ['min', 'max', 'mean', 'std'],
    #'noveltyCount3D': ['min', 'max', 'mean', 'std'],
    #'noveltyCount5D': ['min', 'max', 'mean', 'std'],
    #'noveltyCount7D': ['min', 'max', 'mean', 'std'],
    'volumeCounts12H': ['min', 'max', 'mean', 'std'],
#     'volumeCounts24H': ['min', 'max', 'mean', 'std'],
    #'volumeCounts3D': ['min', 'max', 'mean', 'std'],
    #'volumeCounts5D': ['min', 'max', 'mean', 'std'],
    #'volumeCounts7D': ['min', 'max', 'mean', 'std']
}

In [ ]:
def join_market_news(market_train_df, news_train_df):
    # Fix asset codes (str -> list)
    news_train_df['assetCodes'] = news_train_df['assetCodes'].str.findall(f"'([\w\./]+)'")    
    # Expand assetCodes
    assetCodes_expanded = list(chain(*news_train_df['assetCodes']))
    assetCodes_index = news_train_df.index.repeat( news_train_df['assetCodes'].apply(len) )

    assert len(assetCodes_index) == len(assetCodes_expanded)
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})

    # Create expandaded news (will repeat every assetCodes' row)
    news_cols = ['time', 'assetCodes'] + sorted(news_cols_agg.keys())
    news_train_df_expanded = pd.merge(df_assetCodes, news_train_df[news_cols], left_on='level_0', right_index=True, suffixes=(['','_old']))

    # Free memory
    del news_train_df, df_assetCodes

    # Aggregate numerical news features
    news_train_df_aggregated = news_train_df_expanded.groupby(['time', 'assetCode']).agg(news_cols_agg)
    
    # Free memory
    del news_train_df_expanded

    # Convert to float32 to save memory
    news_train_df_aggregated = news_train_df_aggregated.apply(np.float32)

    # Flat columns
    news_train_df_aggregated.columns = ['_'.join(col).strip() for col in news_train_df_aggregated.columns.values]

    # Join with train
#     market_train_df['diff_open_close'] = (market_train_df['close'] - market_train_df['open'])/market_train_df['open']
    market_train_df
    market_train_df = market_train_df.join(news_train_df_aggregated, on=['time', 'assetCode'])

    # Free memory
    del news_train_df_aggregated
    
    return market_train_df

def join_market_news(market_train_df, news_train_df):    
    return market_train_df

In [ ]:
df = market_train_df[['assetCode', 'universe']].groupby('assetCode').max()
to_drop = df[df.universe==0].index
print(to_drop)
#market_train_df = market_train_df[~market_train_df.assetCode.isin(to_drop)]
# news_train_df = news_train_df[~news_train_df.assetCodes.isin(to_drop)]

In [ ]:
def get_xy(market_train_df, news_train_df, le=None):
    y = market_train_df[market_train_df.time.dt.year>2008]['returnsOpenNextMktres10']#.clip(-1, 1)
    #y = market_train_df['returnsOpenNextMktres10']
    x, le = get_x(market_train_df, news_train_df)
    return x, y, le


def label_encode(series, min_count):
    vc = series.value_counts()
#     le = {c:i for i, c in enumerate(vc.index[vc >= min_count])}
    return {c:i for i, c in enumerate(vc.index[vc >= min_count])}


def get_x2(market_train_df, news_train_df, le=None):
    # Split date into before and after 22h (the time used in train data)
    # E.g: 2007-03-07 23:26:39+00:00 -> 2007-03-08 00:00:00+00:00 (next day)
    #      2009-02-25 21:00:50+00:00 -> 2009-02-25 00:00:00+00:00 (current day)
    news_train_df['time'] = (news_train_df['time'] - np.timedelta64(22,'h')).dt.ceil('1D')

    # Round time of market_train_df to 0h of curret day
    market_train_df['time'] = market_train_df['time'].dt.floor('1D')

    # Join market and news
    x = join_market_news(market_train_df, news_train_df)
    
    # If not label-encoder... encode assetCode
    if le is None:
        le_assetCode = label_encode(x['assetCode'], min_count=10)
        le_assetName = label_encode(x['assetName'], min_count=5)
    else:
        # 'unpack' label encoders
        le_assetCode, le_assetName = le
        
    x['assetCode'] = x['assetCode'].map(le_assetCode).fillna(-1).astype(int)
    x['assetName'] = x['assetName'].map(le_assetName).fillna(-1).astype(int)
    print(x.columns)
    try:
        x.drop(columns=['returnsOpenNextMktres10'], inplace=True)
    except:
        pass
#     try:
#         x.drop(columns=['universe'], inplace=True)
#     except:
#         pass
    x['dayofweek'], x['month'] = x.time.dt.dayofweek, x.time.dt.month
    
    x = x[x.time.dt.year > 2008]
    
    #x.drop(columns='time', inplace=True)
    #x.set_index('time', inplace=True)
    x.drop(columns='assetName', inplace=True)
#    x.fillna(-1000,inplace=True)

    # Fix some mixed-type columns
#     for bogus_col in ['marketCommentary_min', 'marketCommentary_max']:
#         x[bogus_col] = x[bogus_col].astype(float)
    # x = x.fillna(0)
    return x, (le_assetCode, le_assetName)

def get_x(market_train_df, news_train_df, le=None):
    market_train_df['time'] = market_train_df['time'].dt.floor('1D')
    x = market_train_df.copy()
    
    if le is None:
        le_assetCode = label_encode(x['assetCode'], min_count=10)
        le_assetName = label_encode(x['assetName'], min_count=5)
    else:
        le_assetCode, le_assetName = le
        
    x['assetCode'] = x['assetCode'].map(le_assetCode).fillna(-1).astype(int)
    x['assetName'] = x['assetName'].map(le_assetName).fillna(-1).astype(int)
    
    try:
        x.drop(columns=['returnsOpenNextMktres10'], inplace=True)
    except:
        pass
    try:
        x.drop(columns=['universe'], inplace=True)
    except:
        pass
#     x['dayofweek'], x['month'] = x.time.dt.dayofweek, x.time.dt.month
    
    x = x[x.time.dt.year > 2008].drop(columns='assetName', inplace=False)
#     x.drop(columns='assetName', inplace=True)
    return x, (le_assetCode, le_assetName)

In [ ]:
%%time

# This will take some time...
X, y, le = get_xy(market_train_df, news_train_df)

dynamic_X = X.copy()

In [ ]:
X.shape, y.shape

In [ ]:
X.columns

In [ ]:
n_train = int(X.shape[0] * 0.8)
universe = market_train_df[market_train_df.time.dt.year>2008]['universe']
#universe = market_train_df['universe']


time = market_train_df[market_train_df.time.dt.year>2008]['time']
#time = market_train_df['time']
t_valid = time.iloc[n_train:]


# Free memory
#del market_train_df, news_train_df

In [ ]:
len(t_valid)

In [ ]:
u_valid = universe[universe > 0].index
good_universe_index = [x in u_valid for x in t_valid.index]
#t_valid_ = time.iloc[[x for x in t_valid.index if x in u_valid]]
t_valid_ = time.iloc[good_universe_index]

In [ ]:
len(t_valid_)

In [ ]:
X_ = X

In [ ]:
X = X_

In [ ]:
X.columns

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
assets_to_drop = X.groupby('assetCode')[['returnsOpenPrevMktres10']].last()
assets_to_drop = list(assets_to_drop[np.isnan(assets_to_drop.returnsOpenPrevMktres10)].index)

In [ ]:
cluster_X = X[['assetCode', 'time', 'returnsOpenPrevMktres10']].copy()
cluster_X = cluster_X[~cluster_X.assetCode.isin(assets_to_drop)]
cluster_X = cluster_X[cluster_X.assetCode != -1]
cluster_X = cluster_X[cluster_X.time.dt.year > 2015]

In [ ]:
correl = cluster_X.pivot(index='time', columns='assetCode', values='returnsOpenPrevMktres10').dropna(axis=1, thresh=200).corr()
covar = cluster_X.pivot(index='time', columns='assetCode', values='returnsOpenPrevMktres10').dropna(axis=1, thresh=200).cov()


model = SpectralCoclustering(n_clusters=5, random_state=0, svd_method='arpack')
model.fit(correl)

correl_cluster = dict(zip(correl.columns, model.row_labels_))
#print(np.argsort(model.row_labels_))

In [ ]:
cluster_X = X[['assetCode', 'time', 'returnsOpenPrevMktres10']].copy()
cluster_X = cluster_X[~cluster_X.assetCode.isin(assets_to_drop)]
cluster_X = cluster_X[cluster_X.time.dt.year > 2014].dropna()
weights = cluster_X.groupby('assetCode').std()
weights = weights.fillna(weights.median()).returnsOpenPrevMktres10
weights = weights.to_dict()

In [ ]:
#pd.get_dummies(X['assetCode'].map(correl_cluster).fillna(100))

In [ ]:
# import seaborn as sns
# sns.clustermap(to_cluster)

In [ ]:
# from sklearn.cluster.bicluster import SpectralCoclustering
# import matplotlib.pyplot as plt

# model = SpectralCoclustering(n_clusters=18, random_state=0, svd_method='arpack')
# model.fit(to_cluster)

# correl_cluster = dict(zip(to_cluster.columns, model.row_labels_))
# print(np.argsort(model.row_labels_))
# fit_data = to_cluster[np.argsort(model.row_labels_)].T
# fit_data = fit_data[np.argsort(model.column_labels_)].T
# #fit_data = fit_data[:, (np.argsort(model.column_labels_) + 1)]

# plt.matshow(fit_data, cmap=plt.cm.Blues)
# plt.title("Spectral Co-Clustering of Average Correlation Matrix")

# #plt.show()
# #plt.savefig()

# fig = plt.gcf()
# fig.set_size_inches(18.5, 10.5)

In [ ]:
import numba

@numba.jit
def RSI(df, n):
    delta = df.returnsOpenPrevMktres1
    #-----------
    dUp, dDown = delta.copy().fillna(0), delta.copy().fillna(0)
    dUp[dUp <= 0] = 0
    dDown[dDown > 0] = 0
    
    RolUp = dUp.rolling(n).mean()
    RolDown = dDown.rolling(n).mean().abs()

    RS = RolUp / RolDown
    rsi= pd.Series(100.0 - (100.0 / (1.0 + RS)), name='RSI_')
    df = df.join(rsi)
    return df
 
@numba.jit
def kst_oscillator(df, r1=5, r2=10, r3=15, r4=20, n1=10, n2=10, n3=10, n4=15):
    """Calculate KST Oscillator for given data.
    :return: pandas.DataFrame
    """
    serie = df['returnsOpenPrevMktres1'].cumsum()
    M = serie.diff(r1 - 1)
    N = serie.shift(r1 - 1)
    ROC1 = M / N
    M = serie.diff(r2 - 1)
    N = serie.shift(r2 - 1)
    ROC2 = M / N
    M = serie.diff(r3 - 1)
    N = serie.shift(r3 - 1)
    ROC3 = M / N
    M = serie.diff(r4 - 1)
    N = serie.shift(r4 - 1)
    ROC4 = M / N
    KST = pd.Series(
        ROC1.rolling(n1).sum() + ROC2.rolling(n2).sum() * 2 + ROC3.rolling(n3).sum() * 3 + ROC4.rolling(n4).sum() * 4,
        name='KST')
    df = df.join(KST)
    return df

@numba.jit
def true_strength_index(df, r=3, s=6):
    """Calculate True Strength Index (TSI) for given data.
    """
    #M = pd.Series(df['close'].diff(1))
    M = pd.Series(df['returnsOpenPrevMktres1'])
    aM = abs(M)
    EMA1 = pd.Series(M.ewm(span=r, min_periods=r).mean())
    aEMA1 = pd.Series(aM.ewm(span=r, min_periods=r).mean())
    EMA2 = pd.Series(EMA1.ewm(span=s, min_periods=s).mean())
    aEMA2 = pd.Series(aEMA1.ewm(span=s, min_periods=s).mean())
    TSI = pd.Series(EMA2 / aEMA2, name='TSI_')
    df = df.join(TSI)
    return df

@numba.jit
def on_balance_volume(df, n=6):
    """Calculate On-Balance Volume for given data.
    """
    OBV = df.volume.copy()
    OBV = OBV*np.sign(df.returnsOpenPrevMktres1)
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean(), name='OBV_')
    df = df.join(OBV_ma)
    return df

@numba.jit
def force_index(df, n=6):
    """Calculate Force Index for given data.
    """
    F = pd.Series(df['close'].diff(n) * df['volume'].diff(n), name='Force_' + str(n))
    df = df.join(F)
    return df

@numba.jit
def coppock_curve(df, n=8):
    """Calculate Coppock Curve for given data.
    """
    serie = df['returnsOpenPrevMktres1'].cumsum()
    M = serie.diff(int(n * 11 / 10) - 1)
    N = serie.shift(int(n * 11 / 10) - 1)
    ROC1 = M / N
    M = serie.diff(int(n * 14 / 10) - 1)
    N = serie.shift(int(n * 14 / 10) - 1)
    ROC2 = M / N
    Copp = pd.Series((ROC1 + ROC2).ewm(span=n, min_periods=n).mean(), name='Copp')
    df = df.join(Copp)
    return df

@numba.jit
def shifts(df):
    df = df.join(pd.Series(df['returnsOpenPrevMktres1'].shift(1), name='shift_1'))
    df = df.join(pd.Series(df['returnsOpenPrevMktres1'].shift(2), name='shift_2'))
    df = df.join(pd.Series(df['returnsOpenPrevMktres1'].shift(3), name='shift_3'))
    df = df.join(pd.Series(df['returnsOpenPrevMktres1'].shift(4), name='shift_4'))
    return(df)

@numba.jit
def shift_10(df):
    df = df.join(pd.Series(df['returnsOpenPrevMktres10'].shift(10), name='shift_10_1'))
    return(df)

@numba.jit
def shift_10_2(df):
    return(df['returnsOpenPrevMktres10'].shift(10))

@numba.jit
def shift_20(df):
    df = df.join(pd.Series(df['returnsOpenPrevMktres10'].shift(20), name='shift_10_2'))
    return(df)

@numba.jit
def shift_30(df):
    df = df.join(pd.Series(df['returnsOpenPrevMktres10'].shift(30), name='shift_10_3'))
    return(df)

@numba.jit
def macd(df, n_fast, n_slow):
    """Calculate MACD, MACD Signal and MACD difference
    """
    EMAfast = pd.Series(df['returnsOpenPrevMktres1'].ewm(span=n_fast, min_periods=n_slow).mean())
    EMAslow = pd.Series(df['returnsOpenPrevMktres1'].ewm(span=n_slow, min_periods=n_slow).mean())
    MACD = pd.Series(EMAfast - EMAslow, name='MACD_')
    MACDsign = pd.Series(MACD.ewm(span=9, min_periods=9).mean(), name='MACDsign_')
    MACDdiff = pd.Series(MACD - MACDsign, name='MACDdiff_')
    df = df.join(MACD)
    df = df.join(MACDsign)
    df = df.join(MACDdiff)
    return df

@numba.jit
def macd_avg(df, n_fast, n_slow):
    """Calculate MACD, MACD Signal and MACD difference
    """
    EMAfast = pd.Series(df['returnsOpenPrevMktres1'].rolling(n_fast).mean())
    EMAslow = pd.Series(df['returnsOpenPrevMktres1'].rolling(n_slow).mean())
    MACD = pd.Series(EMAfast - EMAslow, name='MACD_')
    MACDsign = pd.Series(MACD.ewm(span=9, min_periods=9).mean(), name='MACDsign_')
    MACDdiff = pd.Series(MACD - MACDsign, name='MACDdiff_')
#     df = df.join(MACD)
#     df = df.join(MACDsign)
    df = df.join(MACDdiff)
    return df

@numba.jit
def macd_avg2(df, n_fast, n_slow):
    """Calculate MACD, MACD Signal and MACD difference
    """
    EMAfast = df['returnsOpenPrevMktres1'].rolling(n_fast).mean()
    EMAslow = df['returnsOpenPrevMktres1'].rolling(n_slow).mean()
    MACD = EMAfast - EMAslow
    MACDsign = MACD.ewm(span=9, min_periods=9).mean()
    MACDdiff = MACD - MACDsign
#     df = df.join(MACD)
#     df = df.join(MACDsign)
    #df = df.join(MACDdiff)
    return MACDdiff

@numba.jit
def ewma(df, n):
    """Calculate MACD, MACD Signal and MACD difference
    """
    EMA = pd.Series(df['returnsOpenPrevMktres1'].ewm(span=n, min_periods=n).mean(), name='ewma')
    df = df.join(EMA)
    return df

@numba.jit
def trix(df, n):
    """Calculate TRIX for given data.
    """
    price = (1+df['returnsOpenPrevMktres1']).cumprod()
    EX1 = price.ewm(span=n, min_periods=n).mean()
    EX2 = EX1.ewm(span=n, min_periods=n).mean()
    EX3 = EX2.ewm(span=n, min_periods=n).mean()
    i = 0
    ROC_l = [np.nan]
    l = len(df.index)
    ROC_l = (EX3 - EX3.shift(1))/EX3.shift(1)
#     while i + 1 < l:#df.index[-1]:
#         ROC = (EX3.iloc[i+1] - EX3.iloc[i]) / EX3.iloc[i]
#         ROC_l.append(ROC)
#         i = i + 1
    Trix = pd.Series(ROC_l, name='Trix_')
    df = df.join(Trix)
    return df


@numba.jit
def std_rol(df, n):
    df = df.join(pd.Series(df['returnsOpenPrevMktres1'].rolling(n).std(), name='std'))
    return(df)

@numba.jit
def stochastic_oscillator_d(df, n):
    """Calculate stochastic oscillator %D for given data.
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    serie = df['returnsOpenPrevMktres1'].cumsum()
    SOk = pd.Series((serie - serie.rolling(10).min()) / (serie.rolling(10).max() - serie.rolling(10).min()), name='SO%k')
    SOd = pd.Series(SOk.ewm(span=n, min_periods=n).mean(), name='SOd')
    df = df.join(SOd)
    return df

In [ ]:
# X[['assetCode', 'returnsClosePrevRaw1']].groupby('assetCode')[['returnsClosePrevRaw1']].apply(RSI)
# volume = X[['assetCode', 'volume', 'close', 'returnsOpenPrevMktres1', 'returnsOpenPrevMktres10']].groupby('assetCode')
# volume.apply(std_rol).dropna()


In [ ]:
%%time 

def add_features(X, clusters=None):
    
#     volume = X[['assetCode', 'volume', 'close']].copy()
#     volume.volume += 10000000
#     volume = volume.groupby('assetCode')
#     X['volume_diff'] = volume[['volume']].pct_change().sort_index()
#     X['volume_MACD_6_3'] = volume[['volume']].pct_change().rolling(3).mean() - volume[['volume']].pct_change().rolling(6).mean()
#     X['copp'] = volume.apply(coppock_curve)[['Copp_6']]
#     X['force'] = volume.apply(force_index)[['Force_6']]
#     X['OBV'] = volume.apply(on_balance_volume)[['OBV_6']]
#     X['TSI'] = volume.apply(true_strength_index)[['TSI_3_6']]
#     X['KST'] = volume.apply(kst_oscillator)[['KST']]
#     del volume
     
    
    #roll_asset = X[['assetCode', 'returnsClosePrevRaw1']].groupby('assetCode')[['returnsClosePrevRaw1']]
    #X['return_close_M6'] = roll_asset.rolling(6).mean().reset_index(0,drop=True).sort_index()
    #X['return_close_M3'] = roll_asset.rolling(3).mean().reset_index(0,drop=True).sort_index()
    #X['return_MACD_6_3'] = X['return_close_M3'] - X['return_close_M6']
    #del roll_asset
#     X = X.loc[:, ['assetCode', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsOpenPrevRaw10', 'volume']]
    
#     roll_mkt = X[['assetCode', 'returnsOpenPrevMktres1', 'returnsOpenPrevMktres10', 'volume']].groupby('assetCode')
#     X['mkt_close_M6'] = roll_mkt[['returnsOpenPrevMktres1']].rolling(6).mean().reset_index(0,drop=True).sort_index()
#     X['mkt_close_M3'] = roll_mkt[['returnsOpenPrevMktres1']].rolling(3).mean().reset_index(0,drop=True).sort_index()
#     X['mkt_MACD_6_3'] = X['mkt_close_M3'] - X['mkt_close_M6']
#     X[['MACD_3_6', 'MACD_3_6_sign', 'MACD_3_6_diff']] = roll_mkt.apply(macd_avg, n_fast=3, n_slow=6)[['MACD_', 'MACDsign_', 'MACDdiff_']]
#     X['mkt_close_M6'] = roll_mkt[['returnsClosePrevMktres1']].rolling(12).mean().reset_index(0,drop=True).sort_index()
#     X['mkt_close_M3'] = roll_mkt[['returnsClosePrevMktres1']].rolling(6).mean().reset_index(0,drop=True).sort_index()
#     X['mkt_MACD_12_6'] = X['mkt_close_M3'] - X['mkt_close_M6']
#     X.loc[:, 'mkt_MACD_6_3'] = roll_mkt[['returnsClosePrevMktres1']].rolling(3).mean().reset_index(0,drop=True).sort_index() - roll_mkt[['returnsClosePrevMktres1']].rolling(6).mean().reset_index(0,drop=True).sort_index() 
#     X.loc[:, 'mkt_close_M24'] = roll_mkt[['returnsClosePrevMktres1']].rolling(24).mean().reset_index(0,drop=True).sort_index()
#     X.loc[:, 'mkt_close_M12'] = roll_mkt[['returnsClosePrevMktres1']].rolling(12).mean().reset_index(0,drop=True).sort_index()
#     X.loc[:, 'mkt_MACD_24_12'] = X['mkt_close_M12'] - X['mkt_close_M24']
#     X[['MACD_12_24', 'MACD_12_24_sign', 'MACD_12_24_diff']] = roll_mkt.apply(macd, n_fast=12, n_slow=24)[['MACD_', 'MACDsign_', 'MACDdiff_']]
#     X[['MACD_3_6', 'MACD_3_6_sign', 'MACD_3_6_diff']] = roll_mkt.apply(macd, n_fast=3, n_slow=6)[['MACD_', 'MACDsign_', 'MACDdiff_']]
#     X.loc[:, 'volume_diff'] = roll_mkt[['volume']].pct_change().sort_index()
    #X.loc[:, 'volume_MACD_6_3'] = roll_mkt[['volume']].pct_change().rolling(3).mean() - roll_mkt[['volume']].pct_change().rolling(6).mean()
#     X['TSI'] = roll_mkt.apply(true_strength_index, r=6, s=12)[['TSI_']]
#     X['RSI'] = roll_mkt.apply(RSI, n=15)[['RSI_']]
#     X['TRIX'] = roll_mkt.apply(trix, n=10)[['Trix_']]
#     X[['OBV']] = roll_mkt.apply(on_balance_volume, n=12)[['OBV_']]
#     X['shift_10_1'] = roll_mkt.apply(shift_10)[['shift_10_1']]
#     X['SOd'] = roll_mkt.apply(stochastic_oscillator_d, n=6)[['SOd']]
#     X['std_10'] = roll_mkt.apply(std_rol, n=10)[['std']]
#     X['KST'] = roll_mkt.apply(kst_oscillator)[['KST']]
#     X['Copp'] = roll_mkt.apply(coppock_curve)[['Copp']]
#     X['std_20'] = roll_mkt.apply(std_rol, n=20)[['std']]
#     X['std_diff'] = X['std_10'] - X['std_20']
#     X['ewma'] = roll_mkt.apply(ewma, n=10)[['ewma']]
#     X['shift_10_2'] = roll_mkt.apply(shift_20)[['shift_10_2']]
#     X.loc[:, 'shift_10_3'] = roll_mkt.apply(shift_30)[['shift_10_3']]
#     del roll_mkt
    
    #roll_news = X[['assetCode', 'sentimentPositive_mean', 'sentimentNegative_mean']].groupby('assetCode')
    #X['pos_ret'] = roll_news[['sentimentPositive_mean']].diff()
    #X['neg_ret'] = roll_news[['sentimentNegative_mean']].diff()
    #X['diff'] = X.returnsOpenPrevRaw10 - X.returnsOpenPrevMktres10
    
    #X['pos_MACD'] = roll_news[['sentimentPositive_mean']].rolling(3).mean().reset_index(0,drop=True).sort_index() - roll_news[['sentimentPositive_mean']].rolling(6).mean().reset_index(0,drop=True).sort_index()
    #X['neg_MACD'] = roll_news[['sentimentNegative_mean']].rolling(3).mean().reset_index(0,drop=True).sort_index() - roll_news[['sentimentNegative_mean']].rolling(6).mean().reset_index(0,drop=True).sort_index()
    
    #X['pos_range'] = X.sentimentPositive_max - X.sentimentPositive_min
    #X['neg_range'] = X.sentimentNegative_max - X.sentimentNegative_min
    #del roll_news
    
    #X = X[['returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'diff_open_close', 'vol_diff', 'return_close_M6']]
#     X = X[['returnsClosePrevRaw1', 'returnsClosePrevMktres1', 'returnsClosePrevRaw10', 'returnsClosePrevMktres10', 'volume_diff', 
#            'mkt_MACD_6_3', 'volume_MACD_6_3', 'sentimentPositive_mean', 'sentimentPositive_max', 
#            'sentimentNegative_mean','sentimentNegative_max', 'volumeCounts24H_mean',
#            'TSI', 'shift_10_1']]
    
    #X = X[['returnsOpenPrevMktres10', 'shift_10_1']]
#     X['neg'] = X['sentimentNegative_median']*X['noveltyCount12H_median']
#     X['pos'] = X['sentimentPositive_median']*X['noveltyCount12H_median']
    
    X = X.loc[:,['time', 'assetCode', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsOpenPrevRaw10', 'open', 'close', 'volume']]
#     X = X.loc[:,['time', 'assetCode', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsOpenPrevRaw10', 'noveltyCount12H_median', 'sentimentNegative_median', 'sentimentPositive_median']]
    
    roll_mkt = X.loc[:,['assetCode', 'returnsOpenPrevMktres1', 'returnsOpenPrevMktres10']].groupby('assetCode')
    X['MACD_3_6_diff'] = roll_mkt.apply(macd_avg, n_fast=3, n_slow=6)[['MACDdiff_']]
    X['shift_10_1'] = roll_mkt.apply(shift_10)[['shift_10_1']]
    
#     X['TSI'] = roll_mkt.apply(true_strength_index, r=6, s=12)[['TSI_']]
#     X['RSI'] = roll_mkt.apply(RSI, n=15)[['RSI_']]
#     X['TRIX'] = roll_mkt.apply(trix, n=10)[['Trix_']]
#     X[['OBV']] = roll_mkt.apply(on_balance_volume, n=12)[['OBV_']]
#     X['SOd'] = roll_mkt.apply(stochastic_oscillator_d, n=6)[['SOd']]
#     X['std_10'] = roll_mkt.apply(std_rol, n=10)[['std']]
#     X['KST'] = roll_mkt.apply(kst_oscillator)[['KST']]
#     X['Copp'] = roll_mkt.apply(coppock_curve)[['Copp']]
#     X['ewma'] = roll_mkt.apply(ewma, n=20)[['ewma']]

    del roll_mkt
    
    X['temp'] = X['returnsOpenPrevRaw10'] * np.sqrt(X['volume'])
    dico = X[['time', 'temp']].groupby('time').mean().to_dict()
    X['mkt_vol'] = X.time.map(dico['temp'])
    
    dico = X[['time', 'returnsOpenPrevMktres1']].groupby('time').mean().to_dict()
    X['mkt_1'] = X.time.map(dico['returnsOpenPrevMktres1'])
    
    dico = X[['time', 'returnsOpenPrevMktres10']].groupby('time').mean().to_dict()
    X['mkt_10'] = X.time.map(dico['returnsOpenPrevMktres10'])
    
#     dico = X[['time', 'returnsOpenPrevMktres10']].groupby('time').std().to_dict()
#     X['mkt_10_std'] = X.time.map(dico['returnsOpenPrevMktres10'])

    X['daily_r'] = (X['close']  - X['open'] )/X['open']
    
    X = X.loc[:, ['time', 'shift_10_1', 'returnsOpenPrevMktres10', 'MACD_3_6_diff', 'returnsOpenPrevRaw10', 'assetCode']]
    
#     X = X.loc[:, ['time', 'shift_10_1', 'returnsOpenPrevMktres10', 'MACD_3_6_diff', 'returnsOpenPrevRaw10', 'assetCode',
#                  'TSI', 'RSI', 'TRIX', 'OBV', 'SOd', 'std_10', 'KST', 'Copp', 'ewma', 'mkt_1', 'mkt_10', 'mkt_vol']]
    
#     X = X.loc[:, ['time', 'assetCode', 'SOd']]
#     X = X.loc[:, ['time', 'assetCode', 'mkt_1', 'mkt_10', 'mkt_vol', 'daily_r']]


    #X = X[['shift_10_1', 'returnsOpenPrevMktres10', 'MACD_3_6_diff', 'assetCode', 'diff']]
    X = X.replace([np.inf, -np.inf], np.nan)
    if clusters is not None:
        #X = pd.concat([X, pd.get_dummies(X['assetCode'].map(correl_cluster).fillna(100))], axis=1)
        volatility = X.assetCode.map(clusters)
        X['volatility'] = np.array(volatility.fillna(volatility.median()))
        #X = pd.concat([X, X['assetCode'].map(correl_cluster)], axis=1)

    return(X)

def add_features_news(X, clusters=None):
    X = X.loc[:,['time', 'assetCode', 'sentimentNegative_min',
       'sentimentNegative_max', 'sentimentNegative_mean',
       'sentimentNegative_std', 'sentimentPositive_min',
       'sentimentPositive_max', 'sentimentPositive_mean',
       'sentimentPositive_std', 'noveltyCount12H_min', 'noveltyCount12H_max',
       'noveltyCount12H_mean', 'noveltyCount12H_std', 'volumeCounts12H_min',
       'volumeCounts12H_max', 'volumeCounts12H_mean', 'volumeCounts12H_std']]
    return(X)



def add_features2(X, clusters=None):
    

    X = X.loc[:,['assetCode', 'returnsOpenPrevMktres1', 'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsOpenPrevRaw10']]
    
    roll_mkt = X.loc[:,['assetCode', 'returnsOpenPrevMktres1', 'returnsOpenPrevMktres10']].groupby('assetCode')
    X.loc[:,'MACD_3_6_diff'] = np.array(roll_mkt.apply(macd_avg, n_fast=3, n_slow=6))#[['MACDdiff_']]

    X.loc[:,'shift_10_1'] = np.array(roll_mkt.apply(shift_10))#[['shift_10_1']]

    del roll_mkt
    
    
    
    #X = X[['returnsOpenPrevMktres10', 'shift_10_1']]
    X = X[['shift_10_1', 'returnsOpenPrevMktres10', 'MACD_3_6_diff', 'returnsOpenPrevRaw10', 'assetCode']]
    #X = X[['shift_10_1', 'returnsOpenPrevMktres10', 'MACD_3_6_diff', 'assetCode', 'diff']]
    
    X = X.replace([np.inf, -np.inf], np.nan)
    
    if clusters is not None:
        #X = pd.concat([X, pd.get_dummies(X['assetCode'].map(correl_cluster).fillna(100))], axis=1)
        volatility = X['assetCode'].map(clusters)
#         volatility = np.array(volatility.fillna(volatility.median()))
        X['volatility'] = np.array(volatility.fillna(volatility.median()))
        #X = pd.concat([X, X['assetCode'].map(correl_cluster)], axis=1)

    return(X)

#asset_codes = X[['assetCode']]
X_ = add_features(X)#, clusters=weights)


In [ ]:
sns.regplot(X_.daily_r.iloc[:100000], y.iloc[:100000])

In [ ]:
X_.tail()

In [ ]:
#X = X.replace([np.inf, -np.inf], np.nan)

In [ ]:
X_.describe()

In [ ]:
X_.corr()

In [ ]:
import seaborn as sns

# sns.regplot(X_.KST[:80000].clip(-20000, 20000), y[:80000])

In [ ]:
n_train = int(X_.shape[0] * 0.8)
X_train, y_train = X_.iloc[:n_train], y.iloc[:n_train]
#X_train, y_train = X_train.loc[X_train.index & u_valid], y_train.loc[y_train.index & u_valid]
X_valid, y_valid = X_.iloc[n_train:], y.iloc[n_train:]
#X_valid, y_valid = X_valid.loc[X_valid.index & u_valid], y_valid.loc[y_valid.index & u_valid]


y_train_size = np.abs(y_train)
y_valid_size = np.abs(y_valid)

y_train = y_train >= 0
y_valid = y_valid >= 0

# y_train = y_train/X_train.volatility
# y_valid = y_valid/X_valid.volatility



In [ ]:
assets_to_drop = X_train.groupby('assetCode')[['returnsOpenPrevMktres10']].last()
assets_to_drop = list(assets_to_drop[np.isnan(assets_to_drop.returnsOpenPrevMktres10)].index)

X_train = X_train[~X_train.assetCode.isin(assets_to_drop)]
y_train = y_train.loc[X_train.index]
y_train_size = y_train_size.loc[X_train.index]

In [ ]:
X_train = X_train.dropna()

train_assets = X_train.assetCode
valid_assets = X_valid.assetCode

# train_time = X_train.time.apply(np.datetime64)
# valid_time = X_valid.time.apply(np.datetime64)

train_time = X_train.time
valid_time = X_valid.time

X_train.drop('assetCode', axis=1, inplace=True)
X_valid.drop('assetCode', axis=1, inplace=True)
X_train.drop('time', axis=1, inplace=True)
X_valid.drop('time', axis=1, inplace=True)

In [ ]:
X_train.shape

In [ ]:
X_valid.shape

In [ ]:
replace_val = X_train.median()

In [ ]:
X_train.head()

In [ ]:
X_train = X_train.dropna()

In [ ]:
X_train.shape

In [ ]:
for col in X_train.columns:
    X_train[col] = X_train[col].fillna(replace_val[col])
    X_valid[col] = X_valid[col].fillna(replace_val[col])

In [ ]:
#X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]
y_train_size = y_train_size.loc[X_train.index]

In [ ]:
#X_train.drop(['assetCode', 'dayofweek', 'month'], axis=1, inplace=True)
#X_valid.drop(['assetCode', 'dayofweek', 'month'], axis=1, inplace=True)

std = StandardScaler()
std.fit(X_train)
X_train = pd.DataFrame(std.transform(X_train), index = X_train.index, columns=X_train.columns)
X_valid = pd.DataFrame(std.transform(X_valid), index = X_valid.index, columns=X_valid.columns)

In [ ]:
X_train['time'] = train_time
X_valid['time'] = valid_time

X_train['assetCode'] = train_assets
X_valid['assetCode'] = valid_assets

In [ ]:
def sigma_score(preds, valid_data, t_valid):
    df_time = t_valid.factorize()[0]#valid_data.params['extra_time']
    labels = valid_data#.get_label()
    
#    assert len(labels) == len(df_time)

    x_t = preds * labels #  * df_valid['universe'] -> Here we take out the 'universe' term because we already keep only those equals to 1.
    
    # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
    # is a pd.Series and call `group_by`
    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()

    return 'sigma_score', score, True

In [ ]:
from scipy.optimize import minimize

def information_ratio_2(w, y_hat, sigma):
#     r = w[0]*y_hat[0] + w[1]*y_hat[1]
    r = np.dot(w.T, y_hat)
#     s = np.sqrt(w[0]*w[0]*epsilon[0]*epsilon[0] + w[1]*w[1]*epsilon[1]*epsilon[1])
    s = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
    #print(np.dot(w.T, np.dot(sigma, w)))
    return -r/s

def compute_w(model, x, covar):
    #cov = covar.loc[x.assetCode, x.assetCode]
    cov = covar.reindex(index = x.assetCode, columns = x.assetCode)
    avg_diag = np.mean([covar.iloc[i, i] for i in range(len(covar))])
    for k in range(len(cov)):
        if np.isnan(cov.iloc[k, k]):
            cov.iloc[k, k] = avg_diag
    cov = cov.fillna(0)
    print(cov.isnull().sum().sum())
    bounds = [(0.2,1) for i in range(x.shape[0])]
    x = x.drop(['time', 'assetCode'], axis=1)
    pred = model.predict(x)-0.5
    res = minimize(
        information_ratio_2,
        np.array([0.6 for i in range(x.shape[0])]),
        args=(pred, cov),
        bounds=bounds, 
        method='L-BFGS-B',
        options = {'maxfun':100}
    )
    print(res.status)
    return(pred*res.x)

def compute_w2(model, x, covar):
    #cov = covar.loc[x.assetCode, x.assetCode]
    cov = covar.reindex(index = x.assetCode, columns = x.assetCode)
    avg_diag = np.mean([covar.iloc[i, i] for i in range(len(covar))])
    for k in range(len(cov)):
        if np.isnan(cov.iloc[k, k]):
            cov.iloc[k, k] = avg_diag
    cov = cov.fillna(0)
    print(cov.isnull().sum().sum())
    bounds = [(0.2,1) for i in range(x.shape[0])]
    x = x.drop(['time', 'assetCode'], axis=1)
    pred = model.predict(x)-0.5
    covar_inv = np.linalg.inv(cov)
    res = np.dot(covar_inv, pred)/np.dot(np.array([1 for i in range(len(pred))]), np.dot(covar_inv, pred))
    print(information_ratio_2(res, pred, cov))
    return(pred*res)
    
def new_pred(model, x, covar):
    result = np.array([])
    for t in x.time.unique():
        print(t)
        result = np.concatenate((result, compute_w(model, x[x.time == t], covar)))
    return(result)

In [ ]:
#model = XGBClassifier(n_jobs=4, n_estimators=40, max_depth=8, eta=0.1, subsample=0.9, colsample_bytree = 0.9, rate_drop=0.05, booster='dart')
#model.fit(X_train, y_train)

#from xgboost import plot_importance
#plot_importance(model)

In [ ]:
#from sklearn.feature_selection import mutual_info_classif, f_classif
#mutual_info_classif(X_train, y_train, n_neighbors = 100)
#f_classif(X_train, y_train)

In [ ]:
model = XGBRegressor(n_jobs=4, n_estimators=32, max_depth=8, min_child_weight=1, eta=0.1, subsample=0.6, rate_drop=0.05, booster='dart', gamma=0.05)

#model = SGDRegressor(max_iter=60)

model.fit(X_train.drop(['time', 'assetCode'], axis=1), y_train)



In [ ]:
model.score(X_train.drop(['time', 'assetCode'], axis=1), y_train)

In [ ]:
model.score(X_valid.drop(['time', 'assetCode'], axis=1), y_valid)

In [ ]:
sns.regplot(y_valid[100000:200000], model.predict(X_valid.drop(['time', 'assetCode'], axis=1))[100000:200000])

In [ ]:
# model_size.coef_

In [ ]:
# sns.regplot(model_size.predict(X_valid)[:50000], y_valid_size[:50000])

In [ ]:
from catboost import CatBoostClassifier
import time

model = CatBoostClassifier(thread_count=4,                            
                            #n_estimators=400, 
                            max_depth=10, 
                            eta=0.1, 
                            loss_function='Logloss', 
                            random_seed = 64738,
                            iterations=100,
                            verbose=10)
model.fit(X_train.drop(['time'], axis=1), y_train, cat_features=[3])

In [ ]:
model.get_feature_importance()

In [ ]:
X_train.columns

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
# model4 = GaussianNB()
# model1 = XGBClassifier(n_jobs=4, n_estimators=40, max_depth=10, eta=0.1, subsample=0.9, colsample_bytree = 0.9, rate_drop=0.05, booster='dart')
model3 = XGBClassifier(n_jobs=4, n_estimators=64, max_depth=11, eta=0.1, subsample=0.5, rate_drop=0.05, booster='dart', verbose=True)
# model4 = KNeighborsClassifier(n_neighbors=50, n_jobs=4)
model1 = XGBClassifier(n_jobs=4, n_estimators=16, max_depth=11, min_child_weight=1, eta=0.1, subsample=0.6, rate_drop=0.05, booster='dart', gamma=0.05)#, eval_metric='auc')
model2 = SGDClassifier(loss='modified_huber', penalty='l1', alpha=0.001, max_iter=30, verbose=10)
#model2 = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.0001, fit_intercept=True, random_state=3, max_iter=400, verbose=0, warm_start=False, n_jobs=1)
# model3 = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.0005, fit_intercept=True, random_state=3, max_iter=400, verbose=0, warm_start=False, n_jobs=1)


model = VotingClassifier([('nb', model1), ('l1', model2)], voting='hard')
#model=KNeighborsClassifier()
model3 = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.01, fit_intercept=True, random_state=3, max_iter=800, verbose=0, warm_start=False, n_jobs=1)
#model = XGBClassifier(n_jobs=4, n_estimators=8, max_depth=10, eta=0.1, subsample=0.9, colsample_bytree = 0.9, rate_drop=0.05, booster='dart', verbose=10)
model = XGBClassifier(n_jobs=4, n_estimators=16, max_depth=11, min_child_weight=1, eta=0.08, subsample=0.6, rate_drop=0.05, booster='dart', gamma=0.05)#, eval_metric='auc')
clf=RandomForestClassifier(n_estimators=4, n_jobs=1, criterion= 'entropy', bootstrap=False, class_weight='balanced_subsample')
clf=XGBClassifier(n_jobs=4, n_estimators=4, max_depth=11, eta=0.1, subsample=1, rate_drop=0.0, booster='dart', verbose=True)
clf=SGDClassifier(loss='modified_huber', penalty='l1', alpha=0.001)
#model = BaggingClassifier(base_estimator=clf, n_estimators=40, max_samples=0.5, n_jobs=4, verbose=10)
#model3.fit(X_train, y_train)
model2=KNeighborsClassifier(n_neighbors=20)
model=XGBClassifier(n_jobs=4, n_estimators=16, max_depth=11, eta=0.1, subsample=0.5, rate_drop=0.05, booster='dart', verbose=True)
# model = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.01, fit_intercept=True, random_state=3, max_iter=1000, verbose=0, warm_start=False, n_jobs=1)
#model = SGDClassifier(loss='modified_huber', penalty='l1', alpha=0.001, max_iter=30, verbose=10)
#model=RandomForestClassifier(n_estimators=32, max_depth=7, n_jobs=4, criterion= 'entropy', bootstrap=False, class_weight='balanced_subsample', verbose=10)
#model = VotingClassifier([('nb', model1), ('l1', model3)], voting='hard')
model.fit(X_train.drop(['time', 'assetCode'], axis=1), y_train)
#model=KNeighborsClassifier(n_neighbors=40)
#model.fit(X_train, y_train)

In [ ]:
from xgboost import plot_importance
plot_importance(model)


In [ ]:
np.unique(X_valid.time)[0]#.tz_localize(None)

In [ ]:
model.coef_

In [ ]:
#pred = new_pred(model, X_valid, covar)

In [ ]:
# df_result = np.array([x[1] for x in model.predict_proba(X_valid)])
# for i in range(len(df_result)):
#     if np.abs(df_result[i] - 0.5) < 0.01:
#         df_result[i] = 0
#     else:
#         df_result[i] = df_result[i] - 0.5
# df_result = (model.predict(X_valid)-0.5)*df_result
#df_result = (model.predict(X_valid)-0.5 + model3.predict(X_valid)-0.5)
df_result = (model.predict(X_valid.drop(['time', 'assetCode'], axis=1))-0.5)

In [ ]:
pred1 = model.predict(X_valid.drop(['time', 'assetCode'], axis=1))

In [ ]:
pred2 = model.predict(X_valid.drop(['time', 'assetCode'], axis=1))

In [ ]:
np.(pred1-0.5)-(pred2-0.5)

In [ ]:
sns.distplot((pred1-0.5)-(pred2-0.5))

In [ ]:
df_result_train = X_train.returnsOpenPrevMktres10 >= 0
df_result_valid = X_valid.returnsOpenPrevMktres10 >= 0

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(df_result_train,y_train))
print(accuracy_score(df_result_valid,y_valid))


In [ ]:
print(accuracy_score(df_result_valid.iloc[good_universe_index],y_valid.iloc[good_universe_index]))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(model.predict(X_train.drop(['time', 'assetCode'], axis=1)),y_train))
print(accuracy_score(model.predict(X_valid.drop(['time', 'assetCode'], axis=1)),y_valid))


In [ ]:
print(accuracy_score(model.predict(X_valid.drop(['time', 'assetCode'], axis=1))[good_universe_index],y_valid.iloc[good_universe_index]))

# Sigma Score

## Signs

In [ ]:
sigma_score(df_result_valid-0.5, y_valid-0.5, t_valid)

In [ ]:
sigma_score((df_result_valid-0.5).iloc[good_universe_index], (y_valid-0.5).iloc[good_universe_index], t_valid_)

In [ ]:
sigma_score(df_result, y_valid-0.5, t_valid)

In [ ]:
sigma_score(df_result[good_universe_index], (y_valid-0.5).iloc[good_universe_index], t_valid_)

In [ ]:
sigma_score([x[1] - 0.5 for x in model.predict_proba(X_valid.drop(['time', 'assetCode'], axis=1))], y_valid-0.5, t_valid)

In [ ]:
sigma_score([x[1] - 0.5 for x in model.predict_proba(X_valid.drop(['time', 'assetCode'], axis=1))[good_universe_index]], (y_valid-0.5).iloc[good_universe_index], t_valid_)

## Test of weighting schemes

In [ ]:
#return_train = y.iloc[:n_train]
#m = min(return_train[return_train > 0].min(), return_train[return_train < 0].max() * -1)

In [ ]:
min(df_result/model_size.predict(X_valid)**8)

In [ ]:
#sigma_score(df_result*(1/X_valid.volatility)*m, y_valid-0.5, t_valid)

In [ ]:
sigma_score(df_result, y.iloc[n_train:], t_valid)

In [ ]:
sigma_score(0.0000001*df_result/model_size.predict(X_valid)**8, y.iloc[n_train:], t_valid)

In [ ]:
pred = (model.predict(X_valid) - 0.5)/((0.01+model_size.predict(X_valid))**8)
sigma_score(np.abs([x[1]-0.5 for x in model.predict_proba(X_valid)])**(0.8)*pred/np.max(np.abs(pred)), y.iloc[n_train:], t_valid)

In [ ]:
sigma_score(df_result, y.iloc[n_train:], t_valid)

In [ ]:
sigma_score(np.array([x[1] - 0.5 for x in model.predict_proba(X_valid)]), y.iloc[n_train:], t_valid)

In [ ]:
sigma_score(np.array([np.sign(x[1] - 0.5) if np.abs(x[1] - 0.5)>0.03 else 0 for x in model.predict_proba(X_valid)]), y.iloc[n_train:], t_valid)

In [ ]:
#sigma_score((np.array([x[1] - 0.5 for x in model.predict_proba(X_valid)]) * (X_valid.volatility)**(1/20))[good_universe_index], y.iloc[n_train:].iloc[good_universe_index], t_valid_)

## Returns

In [ ]:
sigma_score(df_result_valid-0.5, y.iloc[n_train:], t_valid)

In [ ]:
sigma_score((df_result_valid-0.5)[good_universe_index], y.iloc[n_train:].iloc[good_universe_index], t_valid_)

In [ ]:
sigma_score(df_result, y.iloc[n_train:], t_valid)

In [ ]:
sigma_score(df_result[good_universe_index], y.iloc[n_train:].iloc[good_universe_index], t_valid_)

In [ ]:
sigma_score([(x[1] - 0.5) for x in model.predict_proba(X_valid.drop(['time', 'assetCode'], axis=1))], y.iloc[n_train:], t_valid)

In [ ]:
sigma_score([x[1] - 0.5 for x in model.predict_proba(X_valid.drop(['time', 'assetCode'], axis=1))[good_universe_index]], y.iloc[n_train:].iloc[good_universe_index], t_valid_)

In [ ]:
sigma_score(np.array([np.sign(x[1] - 0.5) if np.abs(x[1] - 0.5)>0.02 else np.sign(x[1] - 0.5)/10 for x in model.predict_proba(X_valid.drop(['time', 'assetCode'], axis=1))])[good_universe_index], y.iloc[n_train:].iloc[good_universe_index], t_valid_)

In [ ]:
# import seaborn as sns
sigma_score(pred2-0.5 + 0.04*(pred1-0.5), y.iloc[n_train:], t_valid)

In [ ]:
# for i in range(len(model.coef_[0])):
#     print(X_train.columns[i], model.coef_[0][i])

evals_result = {}
#m = lgb.train(lgb_params, dtrain, num_boost_round=1000, valid_sets=(dvalid,), valid_names=('valid',), verbose_eval=25,
#              early_stopping_rounds=100, feval=sigma_score, evals_result=evals_result)
# model1 = SGDRegressor(loss='huber', penalty='l1', alpha=0.00006, l1_ratio=0.15, fit_intercept=True, max_iter=None, 
#                      tol=None, shuffle=True, verbose=0, epsilon=0.1, random_state=3, learning_rate='invscaling', 
#                      eta0=0.01, power_t=0.25, 
#                      warm_start=False, average=False, n_iter=None)

# model2 = SGDRegressor(loss='epsilon_insensitive', penalty='l2', alpha=0.00006, l1_ratio=0.15, fit_intercept=True, max_iter=None, 
#                      tol=None, shuffle=True, verbose=0, epsilon=0.1, random_state=3, learning_rate='optimal', 
#                      eta0=0.01, power_t=0.25, 
#                      warm_start=True, average=False, n_iter=None)

# model3 = SGDRegressor(loss='squared_epsilon_insensitive', penalty='l1', alpha=0.00006, l1_ratio=0.15, fit_intercept=True, max_iter=None, 
#                      tol=None, shuffle=True, verbose=0, epsilon=0.1, random_state=3, learning_rate='optimal', 
#                      eta0=0.01, power_t=0.25, 
#                      warm_start=False, average=False, n_iter=None)

# model = XGBRegressor(silent=False, base_score=0, max_depth=9, learning_rate=0.1, 
#                       n_estimators=40, nthread=3, gamma=0.2,
#                      min_child_weight=1, subsample=0.9, colsample_bytree = 0.9, rate_drop=0.05, booster='dart')

augmented_X_train = pd.concat([X_train, asset_codes.loc[X_train.index]], axis =1)
assets = set(asset_codes.assetCode)
models = {a:LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.1, fit_intercept=True, 
                               random_state=3, max_iter=800, verbose=0, warm_start=False, n_jobs=1) for a in assets}

model = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=0.0001, fit_intercept=True, random_state=3, max_iter=800, verbose=0, warm_start=False, n_jobs=1)
model.fit(X_train, y_train)

df_result = 0*y_valid.copy()
for a in assets:
    print(a)
    indices = [x for x in asset_codes[asset_codes.assetCode == a].index if x <= X_train.index[-1]]

    indices_test = [x for x in asset_codes[asset_codes.assetCode == a].index if x >= X_valid.index[0]]
    
    if len(indices_test) > 0: 
        if len(indices) > 0:
            new_X = X_train.loc[indices]
            new_y = y_train.loc[indices]
            try:
                models[a].fit(new_X, new_y)

                new_X_valid = X_valid.loc[indices_test]
                df_result.loc[indices_test] = models[a].predict(new_X_valid)
            except ValueError:
                #print(np.unique(new_y))
                #df_result.loc[indices_test] = np.unique(new_y)[0]
                new_X_valid = X_valid.loc[indices_test]
                df_result.loc[indices_test] = model.predict(new_X_valid)
        else:
            new_X_valid = X_valid.loc[indices_test]
            df_result.loc[indices_test] = model.predict(new_X_valid)
    


# model1.fit(X_train, y_train)
# model2.fit(X_train, y_train)
# model3.fit(X_train, y_train)


#df_result = pd.DataFrame(evals_result['valid'])
# df_result = 0.4*model1.predict(X_valid) + 0.3*model2.predict(X_valid) + 0.3*model3.predict(X_valid)
#df_result = 0.2*df_result + 0.8*model1.predict(X_valid)
#df_result = model1.predict(X_valid)

models[1].coef_

model1.score(X_train, y_train)

cols = X.columns
for i in range(len(cols)):
    print(cols[i])
    print(model1.coef_[i])

model1.score(X_valid, y_valid)

from sklearn.metrics import accuracy_score
print(accuracy_score(df_result,y_valid))

sigma_score(df_result-0.5, y_valid-0.5, t_valid)

In [ ]:
#dtrain_full = lgb.Dataset(X, y, feature_name=train_cols, categorical_feature=categorical_cols)
#X = X.drop(['assetCode', 'dayofweek', 'month'], axis=1)



#X_, y = X_.loc[u_valid], y.loc[u_valid]
asset = X_.assetCode
tim = X_.time
X_.drop(['assetCode', 'time'], axis=1, inplace=True)
replace_val = X_.median()
X_ = X_.dropna()
y = y.loc[X_.index]
for col in X_.columns:
    X_[col] = X_[col].fillna(replace_val[col])
#X = X.dropna()


ind_X = X_.index
std = StandardScaler()
std.fit(X_)
X=pd.DataFrame(std.transform(X_), index=X_.index, columns = X_.columns)
X['assetCode'] = asset
X['time'] = tim

In [ ]:
u_valid[-1]

In [ ]:
X.shape

In [ ]:
X_train.shape

In [ ]:
y_size = np.abs(y.loc[y.index & u_valid])
y = y >=0
#model = XGBClassifier(n_jobs=4, n_estimators=100, max_depth=10, eta=0.1, subsample=0.9, colsample_bytree = 0.95, rate_drop=0.05, booster='dart')
#model.fit(X.loc[u_valid], y.loc[u_valid])
#model = XGBClassifier(n_jobs=2, n_estimators=128, max_depth=10, eta=0.1, subsample=0.5, rate_drop=0.05, booster='dart', verbose=True)
model = XGBClassifier(n_jobs=4, n_estimators=64, max_depth=10, eta=0.1, subsample=0.5, rate_drop=0.05, booster='dart', verbose=True)
model.fit(X.drop(['assetCode', 'time'], axis=1), y)

model_size = XGBRegressor(n_jobs=1, n_estimators=1, max_depth=8, min_child_weight=1, eta=0.09, subsample=0.6, rate_drop=0.05, booster='dart', gamma=0.05)
#model_size.fit(X.loc[X.index & u_valid], y_size)

In [ ]:
#sns.regplot(model_size.predict(X)[:50000], y_size[:50000])

In [ ]:
def make_predictions(predictions_template_df, le, model, replace_val, std, x):
#     print(x.isnull().sum().sum())
#     for col in x.columns:
#         x[col] = x[col].fillna(replace_val[col])
#     x = pd.DataFrame(std.transform(x), columns = x.columns, index=x.index)
    #predictions_template_df.confidenceValue = [np.sign(y[1] - 0.5) if np.abs(y[1] - 0.5)>0.05 else 0 for y in model.predict_proba(x)]
    predictions = (model.predict(x) - 0.5)#/((0.01+model_size.predict(x)**2))
    predictions_template_df.confidenceValue = predictions#*(0.01**2)
    
def make_predictions2(predictions_template_df, le, model, replace_val, std, x, weight):
    predictions = (model.predict(x) - 0.5) * weight
    predictions_template_df.confidenceValue = predictions

In [ ]:
def online_learn(model_list, weight_list):
    pass


In [ ]:
days = env.get_prediction_days()

online_train = []
#univ_t = []
x_t = []
# assets_l = []
i=0
# weight=1
# score=0.5
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    i+=1
#     if i % 200 == 199:
#         model.fit(X.iloc[2000*(i//100):].drop(['time', 'assetCode'], axis=1), y.iloc[2000*(i//100):])
        
    x, _ = get_x(market_obs_df, news_obs_df, le)
    #univ_t.append(market_obs_df[['assetCode', 'universe']].copy())
    d = market_obs_df.time[0]
    print(d)
    del news_obs_df
    del market_obs_df
    last_ind = dynamic_X.index[-1]+1
    x.index = [i+last_ind for i in x.index]
    dynamic_X = pd.concat([dynamic_X, x]).iloc[-(15*len(x.index)):]
#     dynamic_X = fast_concat([dynamic_X, x]).iloc[-(15*len(x.index)):]
    x_temp = dynamic_X.copy()
    x_temp = add_features(x_temp)#, clusters=weights)
    #x = x_temp.iloc[-len(x.index):]
    #x = x.drop(['assetCode', 'dayofweek', 'month'], axis=1)
    
    x = x_temp.iloc[-len(x.index):].copy()
    y_online = x[['returnsOpenPrevMktres10', 'assetCode']].dropna().copy()
    online_train.append(x.copy())
    
    x.drop(['assetCode', 'time'], axis=1, inplace=True)
    print(x.isnull().sum().sum())
    for col in x.columns:
        x[col] = x[col].fillna(replace_val[col])
    x = pd.DataFrame(std.transform(x), columns = x.columns, index=x.index)
    
    #update = np.exp(-score+0.5)
#     update = 0.5-score
#     weight = min(max(weight+update, 0.5), 1.5)
    
    make_predictions(predictions_template_df, le, model, replace_val, std, x)
    env.predict(predictions_template_df)
#         assets_l = assets_l[1:]
    if len(online_train)>10:
        x_ = online_train[0]
        x_ = x_[x_.assetCode != -1]
        x_ = x_[x_.assetCode.isin(y_online.assetCode)]
        x_.sort_values('assetCode', inplace=True)
#         print(x_.iloc[:30])
#         univ = univ_t[0]
#         univ = univ[univ.assetCode != -1]
#         univ = univ[univ.assetCode.isin(y_online.assetCode)].sort_values('assetCode')
        y_online = y_online[y_online.assetCode != -1]
        y_online = y_online[y_online.assetCode.isin(x_.assetCode)]
        y_online.sort_values('assetCode', inplace=True)
#         print(y_online.iloc[:30])
        y_online = y_online.returnsOpenPrevMktres10
        y = pd.concat([y, y_online])
        asset = x_['assetCode']
        tim = x_['time']
        x_.drop(['assetCode', 'time'], axis=1, inplace=True)
    
        for col in x_.columns:
            x_[col] = x_[col].fillna(replace_val[col])
        x_ = pd.DataFrame(std.transform(x_), columns = x_.columns, index=x_.index)
        score = model.score(x_, y_online >= 0)
        print(score)
        
        x_t.append(np.sum((model.predict(x_)-0.5)*y_online))
        if len(x_t) >= 2 and d.year <= 2017 or (d.year <= 2018 and d.month<=5):
            temp = np.array(x_t)
            print(np.mean(temp)/np.std(temp))
        x_['assetCode'] = asset
        x_['time'] = tim
        X = pd.concat([X, x_])
#         model.fit(x_, y_online >= 0, xgb_model=model)
#         model.fit(x_, y_online >= 0)
#         sns.regplot(x.returnsOpenPrevMktres10, y_online)
#         plt.show()
        online_train = online_train[1:]
        
#         univ_t = univ_t[1:]
        
        
print('Done!')

In [ ]:
env.write_submission_file()

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(x_t)

In [ ]:
plt.plot(np.cumsum(x_t))

In [ ]:
np.mean(x_t)/np.std(x_t)

In [ ]:
y_t=[np.mean(x_t[:i])/np.std(x_t[:i]) for i in range(len(x_t))]

In [ ]:
plt.plot(y_t[10:])